In [ ]:
import pandas as pd
import numpy as np
from os import path,listdir
from BaselineRemoval import BaselineRemoval
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

In [ ]:
wavenumber=np.array(range(400,4001,4))
dict={}
dict['lambda']=wavenumber
path = r'C:\Users\ks\Desktop\Spectra Identification\IR'
list_files=listdir(path)

num_cleaned=0
failure=[]
for files in list_files:
    df=pd.read_csv(path.join(path,files),header=0)
    Data1=df['x'].to_numpy()
    Data2=df['y'].to_numpy() 
    ind=Data1.argsort() # Sort wavenumber and absorbance
    X=Data1[ind]
    Y=Data2[ind]
    y_new=[]
    
    if X.shape[0] == 0:
        failure.append(files) # Remove empty files
        continue
    if Y.shape[0] == 0:
        failure.append(files) # Remove empty files
        continue
    if X[-1]<=400:
        failure.append(files) # Remove spectra below minimum wavenumber range
        continue
    if X[0]>=4000:
        failure.append(files) # Remove spectra above maximum wavenumber range
        continue
        
    
    Basobj=BaselineRemoval(Y)
    
    try:
     
        Y=Basobj.ZhangFit()
        #Y=Basobj.IModPoly(degree =3)
    except:
        failure.append(files)
        continue
    Y=savgol_filter(Y,19,3)
# Rest of the code finds the wavenumber in data closest to each wavenumber in chosen range at chosen resolution and maps the absorbances
    if X[0] <= 400:
        if X[-1]<4000:
            dummy=np.square(wavenumber-X[-1])
            index2=np.argmin(dummy)
        else:
            index2=900
            
        for w in wavenumber[:index2+1]:
            dummy= np.square(X-w)
            ind=np.argmin(dummy)
            y_new.append(Y[ind])
        if index2!=900:
            for i in range(index2+1,901):
                y_new.append(0)
                
    if X[0]>400:
        dummy=np.square(wavenumber-X[0])
        index=np.argmin(dummy)
        if X[-1]<4000:
            dummy=np.square(wavenumber-X[-1])
            index2=np.argmin(dummy)
        else:
            index2=900
        y_new=[0]*(index)
        for w in wavenumber[index:index2+1]:
            dummy= np.square(X-w)
            ind=np.argmin(dummy)
            y_new.append(Y[ind])
        if index2!=900:
            for i in range(index2+1,901):
                y_new.append(0)

    dict[files]=y_new
    num_cleaned+=1
    
    print('Number Cleaned:{}. Length of y:{}'.format(num_cleaned,len(y_new)))
    
    
df=pd.DataFrame(dict)
df.to_csv(path.join(path,'BaselineCorrected'+'.csv'))
    

In [ ]:
dict

In [ ]:
df=pd.DataFrame(dict)
df.to_csv('Bio_5comp_Baseline.csv')


In [ ]:
for f in failure:
    with open(r'C:\Users\Karthik\Desktop\Spectra Identification\Clean_Failure_baseline.txt','a')  as file:
            content = '{}\n'.format(f)
            file.write(content)

In [ ]:
failure